In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=dc20a469db83581a502e50a646a356608a4a6b29889cc953ffa427743dffb47a
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [ ]:
import glob
import numpy as np

from IPython.display import HTML
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.animation import FuncAnimation



In [ ]:
DATA_PATH="/content/drive/MyDrive/Colab Notebooks/Chapter 7 - Dynamic Graphs/35"

files = glob.glob(f"{DATA_PATH}/*.npy")
files.sort()
files

edges = np.load(f'{DATA_PATH}/edges.npy')


# Create an adjacency matrix of zeros
num_nodes = edges.max() + 1  # Assuming nodes are 0-indexed
adj = np.zeros((num_nodes, num_nodes))

# Fill in the entries in the adjacency matrix corresponding to edges
for i, j in edges:
    adj[i, j] = 1


In [ ]:
pos =  np.load(files[3])
pos.shape # batchsize x T x nodes x D

(92, 50, 31, 3)

In [ ]:
edges = np.load(files[0])


In [ ]:
from torch_geometric.data import Data
from torch.utils.data import Dataset
import numpy as np
import torch

def normalize_array(arr):
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    normalized_arr = (arr - arr_min) / (arr_max - arr_min)
    return normalized_arr, arr_min, arr_max

def inverse_normalize_array(normalized_arr, arr_min, arr_max):
    denormalized_arr = normalized_arr * (arr_max - arr_min) + arr_min
    return denormalized_arr


from torch_geometric.data import Data
from torch.utils.data import Dataset
import numpy as np
import torch

def normalize_array(arr):
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    normalized_arr = (arr - arr_min) / (arr_max - arr_min)
    return normalized_arr, arr_min, arr_max

def inverse_normalize_array(normalized_arr, arr_min, arr_max):
    denormalized_arr = normalized_arr * (arr_max - arr_min) + arr_min
    return denormalized_arr

class PoseDataset(Dataset):
    def __init__(self, loc_path, vel_path, edge_path, mask_path, mask_size,
                 max_size=50,
                 transform=True):

       # Load the data from .npy files
        self.locations = np.load(loc_path)
        self.velocities = np.load(vel_path)

        self.transform=transform

        self.edges = np.load(edge_path)
        self.masks = np.load(mask_path)
        self.mask_size = mask_size
        self.window_size = max_size - self.mask_size
        self.temporal = False

    def __len__(self):
        # The length of the dataset is the number of windows in each run
        return self.locations.shape[0] * (self.locations.shape[1] - self.window_size + 1)

    def __getitem__(self, idx):
        total_timesteps = self.window_size + self.mask_size
        # Concatenate location and velocity data for each node
        nodes = np.concatenate((self.locations[idx][:total_timesteps], self.velocities[idx][:total_timesteps]), axis=2)
        nodes = nodes.reshape(-1, nodes.shape[-1])

        # Apply normalization if transform is True
        if self.transform:
            nodes, node_min, node_max = normalize_array(nodes)

        # Repeat the edges for the total number of timesteps (past + future)
        edge_index = np.repeat(self.edges[None, :], total_timesteps, axis=0)

        # Apply the shift to the edge indices
        shift = np.arange(total_timesteps)[:, None, None] * self.locations.shape[2]
        edge_index += shift
        edge_index = edge_index.reshape(2, -1)  # Flatten the edge indices into two dimensions

        # Convert everything to PyTorch tensors
        x = torch.tensor(nodes, dtype=torch.float)
        edge_index = torch.tensor(edge_index, dtype=torch.long)

        # Check edge index values
        assert edge_index.max().item() < x.size(0), f"Maximum node index in edge_index ({edge_index.max()}) exceeds the total number of nodes ({x.shape[0]})"

        # Calculate the indices of the masked nodes
        mask_indices = np.arange(self.window_size * self.locations.shape[2], total_timesteps * self.locations.shape[2])
        mask_indices = torch.tensor(mask_indices, dtype=torch.long)

        if self.transform:
            trnsfm_data = [node_min, node_max]
            return Data(x=x, edge_index=edge_index, mask_indices=mask_indices, y=x, trnsfm=trnsfm_data)

        return Data(x=x, edge_index=edge_index, mask_indices=mask_indices, y=x)


In [ ]:
from torch_geometric.loader import DataLoader

batch_size = 1
mask_size=10
train_dataset = PoseDataset(
    f'{DATA_PATH}/loc_train_cmu.npy',
    f'{DATA_PATH}/vel_train_cmu.npy',
    f'{DATA_PATH}/edges.npy',
    f'{DATA_PATH}/joint_masks.npy',
    mask_size = mask_size,
    transform=True
  )

val_dataset = PoseDataset(
    f'{DATA_PATH}/loc_valid_cmu.npy',
    f'{DATA_PATH}/vel_valid_cmu.npy',
    f'{DATA_PATH}/edges.npy',
    f'{DATA_PATH}/joint_masks.npy',
    mask_size = mask_size,
    transform=True
  )

test_dataset = PoseDataset(
    f'{DATA_PATH}/loc_test_cmu.npy',
    f'{DATA_PATH}/vel_test_cmu.npy',
    f'{DATA_PATH}/edges.npy',
    f'{DATA_PATH}/joint_masks.npy',
    mask_size = mask_size,
    transform=True
  )


In [ ]:
test_dataset[0]

Data(x=[1550, 6], edge_index=[2, 1500], y=[1550, 6], mask_indices=[310], trnsfm=[2])

In [ ]:
import torch
from torch import nn
from torch_geometric.nn import GATv2Conv

class GAT(torch.nn.Module):
    def __init__(self, n_feat,
                 hidden_size=32,
                 num_layers=3,
                 num_heads=1,
                 dropout=0.2,
                 mask_size=10):
        super(GAT, self).__init__()

        self.num_layers = num_layers
        self.heads = num_heads
        self.n_feat = n_feat
        self.hidden_size = hidden_size
        self.gat_layers = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        self.dropout = nn.Dropout(dropout)
        self.mask_size = mask_size

        # First GAT Layer
        self.gat_layers.append(GATv2Conv(self.n_feat, self.hidden_size, heads=num_heads))
        # BatchNorm Layer for the first GAT Layer
        self.batch_norms.append(nn.BatchNorm1d(num_features=self.hidden_size * num_heads))

        # Intermediate GAT Layers
        for _ in range(num_layers-2):
          self.gat_layers.append(GATv2Conv(self.hidden_size*num_heads, self.hidden_size, heads=num_heads))
          # BatchNorm Layers for intermediate GAT Layers
          self.batch_norms.append(nn.BatchNorm1d(num_features=self.hidden_size * num_heads))

        # Last GAT Layer
        self.gat_layers.append(GATv2Conv(self.hidden_size * num_heads, self.n_feat))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for i in range(self.num_layers):
            x = self.gat_layers[i](x, edge_index)
            if i < self.num_layers - 1:
                x = self.batch_norms[i](x)
                x = torch.relu(x)
                x = self.dropout(x)

        # Only output the last frame
        n_nodes = edge_index.max().item() + 1
        x = x.view(-1, n_nodes, self.n_feat)
        return x[-self.mask_size:].view(-1, self.n_feat)


In [ ]:
from torch.nn import MSELoss
from torch.optim import Adam

# Instantiate the model
model = GAT(n_feat = 6, num_heads=3,hidden_size=128, num_layers=3, mask_size=10)

# Define loss function and optimizer
criterion = MSELoss()
optimizer = Adam(model.parameters(), lr=0.0005)

# Number of epochs
epochs = 50


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [ ]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(epochs), ncols=300):
    model.train()
    train_loss = 0.0
    for data in train_dataset:
        optimizer.zero_grad()
        # Generate the model's predictions for the input
        out = model(data)
        # Compute the loss between the outputs and the targets
        loss = criterion(out, data.y.reshape(out.shape[0], -1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_data in val_dataset:
            # Generate the model's predictions for the input
            val_out = model(val_data)
            # Compute the loss between the outputs and the targets
            val_loss += criterion(out, data.y.reshape(out.shape[0], -1)).item()

    val_loss /= len(val_dataset)
    train_loss /= len(train_dataset)

    print(f"Epoch {epoch+1}/{epochs}.. Training loss: {train_loss}.. Validation loss: {val_loss}")


  0%|                                                                                                         …

Epoch 1/50.. Training loss: 0.029207972949554798.. Validation loss: 0.018411076881668785
Epoch 2/50.. Training loss: 0.015508389733584501.. Validation loss: 0.013656948100436817


In [ ]:
test_loss = 0
for test_data in test_dataset:
    # Generate the model's predictions for the input
    test_out = model(test_data)
    # Compute the loss between the outputs and the targets
    test_loss += criterion(out, data.y.reshape(out.shape[0], -1)).item()

print(test_loss)

In [ ]:
test_loss

In [ ]:
test_d = test_dataset[0]
# reshaped_pred_data.shape

In [ ]:
model(test_d).detach().numpy().shape

In [ ]:
test_d.y.cpu().numpy().shape

In [ ]:
def reshape_data(data, num_frames=40, num_nodes=31, num_features=6):

    # Check if the raw data can be reshaped to the desired dimensions
    if data.shape[0] != num_frames * num_nodes:
        raise ValueError('The size of the raw data does not match the provided dimensions.')

    # Reshape the data
    reshaped_data = data.reshape(num_frames, num_nodes, num_features)

    return reshaped_data

total_frames = 50
# Create a copy of reshaped_true_data before doing the swap operation
true_data_og = reshape_data(test_d.x.cpu().numpy(), num_frames=total_frames)
reshaped_true_data = np.copy(true_data_og)
reshaped_true_data[:,:, [1,2]] = reshaped_true_data[:,:, [2,1]]

# # Similarly for reshaped_test_data
test_data_og = reshape_data(test_d.y.cpu().numpy(), num_frames=total_frames)
reshaped_test_data = np.copy(test_data_og)
reshaped_test_data[:,:, [1,2]] = reshaped_test_data[:,:, [2,1]]

# Similarly for reshaped_test_data
pred_data_og = reshape_data(model(test_d).detach().numpy(), num_frames=total_frames)
reshaped_pred_data = np.copy(pred_data_og)
reshaped_pred_data[:,:, [1,2]] = reshaped_pred_data[:,:, [2,1]]



In [ ]:
reshaped_true_data.shape

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), frameon=False)

sensor_data = pred_data_og[0,:,1:3]
positions = {i: (sensor_data[i, 0], sensor_data[i, 1]) for i in range(sensor_data.shape[0])}

G = nx.Graph()
G.add_edges_from(edges)

nx.draw_networkx_nodes(G, positions, ax=ax, node_color='k', alpha=0.5)
nx.draw_networkx_edges(G, positions, ax=ax, edge_color='k')

ax.set_axis_on()  # turn the axis on

x_values = [coord[0] for coord in positions.values()]
y_values = [coord[1] for coord in positions.values()]

ax.set_xticks(np.linspace(min(x_values), max(x_values), 6))  # 6 ticks along x-axis
ax.set_yticks(np.linspace(min(y_values), max(y_values), 6))  # 6 ticks along y-axis

plt.show()


In [ ]:
reshaped_pred_data[0][0],reshaped_test_data[0][0]

In [ ]:
edges = np.load(files[0])

fig, axes = plt.subplots(1, mask_size, figsize=(50, 10), frameon=False)
plt.subplots_adjust(wspace=0)

for n in range(mask_size):
    sensor_data = reshaped_test_data[n][:, 1:3]

    sensor_data_pred = reshaped_pred_data[n][:, 1:3]
    # Create an empty graph
    G = nx.Graph()

    # Add edges to the graph
    G.add_edges_from(edges)

    # Visualize the graph using NetworkX
    ax = axes[n]
    ax.set_axis_off()
    # nx.draw_networkx_nodes(G, sensor_data, ax=ax, node_color='k', alpha=(n+1)/10)
    # nx.draw_networkx_edges(G, sensor_data, ax=ax, edge_color='k')

    nx.draw_networkx_nodes(G, sensor_data_pred, ax=ax, node_color='k', alpha=(n+1)/10)
    nx.draw_networkx_edges(G, sensor_data_pred, ax=ax, edge_color='k')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig = plt.figure(figsize=(2, 5))
ax = fig.add_subplot(111)

# sensor_data = data_pred[0][:, 1:]
sensor_data_pred = reshaped_pred_data[0][:, 1:3]
# sensor_data_pred[:, [0, 1]] = sensor_data_pred[:, [1, 0]]
# sensor_data[:,] = sensor_data[:, [1, 0]]

# Create a graph object
G = nx.Graph()

# Add edges to the graph
G.add_edges_from(edges)

# Initialize empty node and edge collections for data_true
# node_collection_true = nx.draw_networkx_nodes(G, sensor_data_true, ax=ax, node_size=50, node_color='black')
# edge_collection_true = nx.draw_networkx_edges(G, sensor_data_true, ax=ax, edge_color='black')

# Initialize empty node and edge collections for data_pred
node_collection_pred = nx.draw_networkx_nodes(G, sensor_data_pred, ax=ax, node_size=50, node_color='red')
edge_collection_pred = nx.draw_networkx_edges(G, sensor_data_pred, ax=ax, edge_color='red')

# Function to update the graph animation
def update(frame):
  sensor_data_pred = reshaped_pred_data[frame][:, 1:3]
  # sensor_data_pred[:, [0, 1]] = sensor_data_pred[:, [1, 0]]

  # Update node and edge collections for data_true
  # node_collection_true.set_offsets(sensor_data_true)
  # edge_collection_true.set_segments([sensor_data_true[edge, :] for edge in G.edges()])

  # Update node and edge collections for data_pred
  node_collection_pred.set_offsets(sensor_data_pred)
  edge_collection_pred.set_segments([sensor_data_pred[edge, :] for edge in G.edges()])

  # return node_collection_true, edge_collection_true, node_collection_pred, edge_collection_pred
  return node_collection_pred, edge_collection_pred

# ax.set_xlim(35, 48)

# Create the animation
animation = FuncAnimation(fig, update, frames=range(mask_size), interval=50, blit=True)
HTML(animation.to_jshtml())
